In [1]:
# Fichier : 13.modeling_chatbot_proxy_model.ipynb
# VERSION FINALE ET DÉFINITIVE

import pandas as pd
import lightgbm as lgb
import joblib
import os

print("--- Entraînement du Modèle Proxy Final pour le Chatbot ---")

# --- 1. Chargement des données et du modèle expert ---
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
model_expert_path = "../../models/model_risk_of_default.pkl"
model_expert_cols_path = "../../models/model_1_columns.pkl"

df = pd.read_parquet(input_path)
model_expert = joblib.load(model_expert_path)
model_expert_columns = joblib.load(model_expert_cols_path)
print("✅ Données et modèle expert chargés.")


# --- 2. Calcul des scores "vérité" avec le modèle expert ---
# On fait le feature engineering nécessaire pour le modèle expert
df['issue_d'] = pd.to_datetime(df['issue_d'])
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['credit_history_length'] = (df['issue_d'] - df['earliest_cr_line']).dt.days

# Préparation des données complètes pour l'expert
features_to_exclude = [
    'loan_status', 'is_default', 'id', 'grade', 'sub_grade', 'int_rate', 'installment', 'last_pymnt_d',
    'emp_title', 'emp_title_clean', 'title', 'zip_code', 'job_category', 'csp_category', 
    'issue_d', 'earliest_cr_line'
]
X_expert = df.drop(columns=features_to_exclude, errors='ignore')
X_expert = pd.get_dummies(X_expert, drop_first=True, dtype=float)
X_expert = X_expert.reindex(columns=model_expert_columns, fill_value=0)

y_target = model_expert.predict_proba(X_expert)[:, 1]
print("✅ Cible (scores de l'expert) générée.")


# --- 3. Préparation des features pour le modèle proxy (doit refléter l'app) ---
print("Préparation des features pour le proxy...")
# On crée les ratios, car l'app les calcule aussi
df['loan_to_income_ratio'] = df['loan_amnt'] / df['annual_inc']
df['loan_to_income_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)
df['revolving_to_income_ratio'] = df['revol_bal'] / df['annual_inc']
df['revolving_to_income_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)

# Sélection des features EXACTES du formulaire Streamlit
features_for_proxy = [
    'loan_amnt', 'annual_inc', 'dti', 'revol_bal', 'revol_util', 'emp_length',
    'home_ownership', 'purpose', 'total_acc', 'open_acc', 'mort_acc', 'pub_rec',
    'pub_rec_bankruptcies', 'credit_history_length', 'loan_to_income_ratio',
    'revolving_to_income_ratio'
]
df['term'] = 36.0 # Champ fixe dans l'app
df['verification_status'] = 'Verified' # Champ fixe dans l'app
features_for_proxy.extend(['term', 'verification_status'])

X_proxy = df[features_for_proxy]
X_proxy = pd.get_dummies(X_proxy, drop_first=True, dtype=float)


# --- 4. Entraînement et Sauvegarde du modèle proxy final ---
proxy_model = lgb.LGBMRegressor(random_state=42)
print("Entraînement du modèle proxy final...")
proxy_model.fit(X_proxy, y_target)
print("✅ Modèle proxy final entraîné.")

proxy_model_path = "../../models/chatbot_proxy_model.pkl"
proxy_model_cols_path = "../../models/chatbot_proxy_model_columns.pkl"
joblib.dump(proxy_model, proxy_model_path)
joblib.dump(X_proxy.columns.tolist(), proxy_model_cols_path)
print(f"✅ Modèle proxy final et ses colonnes sauvegardés.")

--- Entraînement du Modèle Proxy Final pour le Chatbot ---
✅ Données et modèle expert chargés.
✅ Cible (scores de l'expert) générée.
Préparation des features pour le proxy...
Entraînement du modèle proxy final...


C:\Users\Pierre\AppData\Local\Temp\ipykernel_15716\4226198157.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['loan_to_income_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_15716\4226198157.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

✅ Modèle proxy final entraîné.
✅ Modèle proxy final et ses colonnes sauvegardés.
